<a href="https://colab.research.google.com/github/Ananya-1233/Sign-Language-Recognition/blob/main/SignLanguageRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
import random
import cv2

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df.head()

,uid,gloss,duration,category
0,--7-rCNOiK0_1,branch,6.0,flora and fauna
1,-02y2VaVBmw,divorce petition,11.0,legal
2,-03Dju3yPHE,air conditioner,5.0,machine
3,-0Oo1kPonYQ,stapoo,6.0,sport
4,-0XujLX2_9I_2,application,6.0,action


In [ ]:
df = df[:1000]

In [ ]:
df.shape

(1000, 4)

In [ ]:
gloss = df['gloss'].to_list()

In [ ]:
mean = df['duration'].mean()
mean

6.787

In [ ]:
df['duration'].value_counts()

4.0     180
6.0     155
5.0     151
7.0     139
8.0      94
9.0      67
3.0      55
10.0     49
11.0     38
15.0     21
14.0     19
13.0     16
12.0     16
Name: duration, dtype: int64

In [ ]:
prefix_loc = '/content/drive/MyDrive/CISLR/CISLR_v1.5-a_videos/'
suffix_text = '.mp4'
df['uid'] = df['uid'].apply(lambda x: f"{prefix_loc}{x}")
df['uid'] = df['uid'].apply(lambda x: f'{x}{suffix_text}')
df.head()

,uid,gloss,duration,category
0,/content/drive/MyDrive/CISLR/CISLR_v1.5-a_vide...,branch,6.0,flora and fauna
1,/content/drive/MyDrive/CISLR/CISLR_v1.5-a_vide...,divorce petition,11.0,legal
2,/content/drive/MyDrive/CISLR/CISLR_v1.5-a_vide...,air conditioner,5.0,machine
3,/content/drive/MyDrive/CISLR/CISLR_v1.5-a_vide...,stapoo,6.0,sport
4,/content/drive/MyDrive/CISLR/CISLR_v1.5-a_vide...,application,6.0,action


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   uid       1000 non-null   object 
 1   gloss     1000 non-null   object 
 2   duration  1000 non-null   float64
 3   category  1000 non-null   object 
dtypes: float64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
videos = df['uid'].to_list()
labels = df['gloss'].to_list()
videos[0] , labels[-5:]

('/content/drive/MyDrive/CISLR/CISLR_v1.5-a_videos//content/drive/MyDrive/CISLR/CISLR_v1.5-a_videos//content/drive/MyDrive/CISLR/CISLR_v1.5-a_videos/--7-rCNOiK0_1.mp4.mp4.mp4',
 ['your', 'press', 'papaya', 'semi urban', 'juhu beach'])

In [ ]:
data = tuple(zip(videos , labels))

In [ ]:
len(videos) , len(labels)

(1000, 1000)

In [ ]:
for i, element in enumerate(labels):
    if not isinstance(element, str):
        labels[i] = str(element)

In [ ]:
import re

def remove_symbols(text):
    # Define a regex pattern to match symbols
    pattern = re.compile('[^a-zA-Z0-9\s]')

    # Use sub() method to replace symbols with an empty string
    result = pattern.sub('', text)

    return result

# Example usage
output_strings = [remove_symbols(string) for string in labels]

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 10
NUM_FEATURES = 2048

In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=5, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


In [ ]:
def prepare_all_videos(df, root_dir, labels = labels):
    num_samples = len(df)
    video_paths = videos
    labels = labels
    #labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(path)
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(df, "train")
test_data, test_labels = prepare_all_videos(df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")


Frame features in train set: (1000, 10, 2048)


In [ ]:
train_data = list(train_data)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(train_labels)
encoded_labels = label_encoder.transform(train_labels)
labels = to_categorical(encoded_labels)

In [ ]:
len(labels)

1000

In [ ]:
def get_sequence_model():

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(256, return_sequences=True)(frame_features_input, mask = mask_input)
    x = keras.layers.GRU(128, return_sequences=True)(x)
    x = keras.layers.GRU(64)(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dropout(0.2)(x)
    output = keras.layers.Dense(924, activation="softmax")(x)
    input = [frame_features_input, mask_input]
    rnn_model = keras.Model(input, output)

    rnn_model.compile(
        loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate = 5), metrics=["accuracy"]
    )
    return rnn_model


# # data = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
# train_data_0 = tf.data.Dataset.from_tensor_slices(train_data[0])
# train_data_1 = tf.data.Dataset.from_tensor_slices(train_data[1])
def run_experiment():

    seq_model = get_sequence_model()
    history = seq_model.fit(train_data,
                            labels,
                            batch_size = 64,
                            epochs=100,
                            )

    # _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    # print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/100
16/16 [==============================] - 16s 81ms/step - loss: 20.6077 - accuracy: 0.0010
Epoch 2/100
16/16 [==============================] - 1s 80ms/step - loss: 13.0538 - accuracy: 0.0000e+00
Epoch 3/100
16/16 [==============================] - 1s 79ms/step - loss: 11.7645 - accuracy: 0.0000e+00
Epoch 4/100
16/16 [==============================] - 1s 84ms/step - loss: 11.6431 - accuracy: 0.0010
Epoch 5/100
16/16 [==============================] - 1s 85ms/step - loss: 11.5675 - accuracy: 0.0010
Epoch 6/100
16/16 [==============================] - 2s 153ms/step - loss: 11.4925 - accuracy: 0.0000e+00
Epoch 7/100
16/16 [==============================] - 2s 109ms/step - loss: 11.4662 - accuracy: 0.0000e+00
Epoch 8/100
16/16 [==============================] - 1s 82ms/step - loss: 11.4655 - accuracy: 0.0000e+00
Epoch 9/100
16/16 [==============================] - 1s 80ms/step - loss: 11.3401 - accuracy: 0.0010
Epoch 10/100
16/16 [==============================] - 1s 81ms/step -

In [ ]:
sequence_model.evaluate(train_data, labels)